In [1]:
import xarray as xr
import numpy as np

In [2]:
fp = "/beegfs/CMIP6/jdpaul3/cmip6_indicators_for_rasdaman/cmip6_indicators_ensemble.nc"
ds = xr.open_dataset(fp, decode_cf=True)

In [3]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:      (model: 14, scenario: 5, time: 151, lat: 43, lon: 288)
Coordinates:
  * lat          (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
  * lon          (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * scenario     (scenario) int64 40B 0 1 2 3 4
  * model        (model) int64 112B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * time         (time) datetime64[ns] 1kB 1950-01-01 1951-01-01 ... 2100-01-01
Data variables:
    rx1day       (model, scenario, time, lat, lon) float64 1GB ...
    dw           (model, scenario, time, lat, lon) float64 1GB ...
    su           (model, scenario, time, lat, lon) float64 1GB ...
    ftc          (model, scenario, time, lat, lon) float64 1GB ...
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    title:        CMIP6 Climate Indicator Data on a Common Grid with Multi-Mo...
    description:  Climate Indicator data from 13 CMIP6 models on a common gri...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    contact:      uaf-snap-data-tools@alaska.edu
    url:          https://uaf-snap.org/

In [4]:
# test each indicator to make sure the ensemble mean is calculated correctly

# historical data

lat = 65
lon = -142
time = "2014-01-01T00:00:00.000000000"
scenario = 0
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Indicator: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Indicator: rx1day
Original Dataset Values:
[16.92398787 15.11431527 10.63752025 18.90382902 11.24431514 20.75390825
 31.54338654 15.50828945 16.29130924 13.1966054  18.34404152 21.86968373
 22.06943622 17.87697138]

 Calculated Ensemble Mean:
17.876971376916536
Dataset Ensemble Mean:
17.876971376916536



Indicator: dw
Original Dataset Values:
[        nan 62.         18.         36.         28.         56.
 82.         44.         13.         21.         39.         22.
 28.         37.41666667]

 Calculated Ensemble Mean:
37.41666666666667
Dataset Ensemble Mean:
37.416666666666664



Indicator: su
Original Dataset Values:
[        nan  0.          0.          0.          0.          0.
  0.          4.         12.          0.          0.          0.
 19.          2.91666667]

 Calculated Ensemble Mean:
2.9166666666666665
Dataset Ensemble Mean:
2.9166666666666665



Indicator: ftc
Original Dataset Values:
[ nan 77.  89.  73.  84.  37.  56.  36.  46.  51.  98.  82.  81.  67.5]

 Calcul

In [5]:
# test each indicator to make sure the ensemble mean is calculated correctly

# scenario data

lat = 65
lon = -142
time = "2020-01-01T00:00:00.000000000"
scenario = 1
models = slice(0, 13)  # first 13 models


for var in list(ds.data_vars):
    if var != "spatial_ref":
        print(f"Indicator: {var}")

        print("Original Dataset Values:")
        values = (
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=models)
            .values
        )

        print(values)
        print("\n Calculated Ensemble Mean:")
        print(np.nanmean(values))

        print("Dataset Ensemble Mean:")
        print(
            ds[var]
            .sel(
                lat=lat,
                lon=lon,
                method="nearest",
            )
            .sel(time=time, scenario=scenario, model=13)
            .values
        )

        print("\n\n")

Indicator: rx1day
Original Dataset Values:
[31.42146757 15.09029232         nan 18.34832383 14.21948508 38.4673859
 13.82481311 15.1396106  17.0270815  14.9401322  23.93052299 23.06017466
 20.87512566 20.52870129]

 Calculated Ensemble Mean:
20.528701285365965
Dataset Ensemble Mean:
20.528701285365965



Indicator: dw
Original Dataset Values:
[18. 77. nan 34. 22. 59. 56. 18.  3. 43. 14. 13. 63. 35.]

 Calculated Ensemble Mean:
35.0
Dataset Ensemble Mean:
35.0



Indicator: su
Original Dataset Values:
[ 2.          0.                 nan         nan  0.          0.
  0.          0.          4.          0.          3.          0.
 12.          1.90909091]

 Calculated Ensemble Mean:
1.9090909090909092
Dataset Ensemble Mean:
1.9090909090909092



Indicator: ftc
Original Dataset Values:
[76.         72.                 nan         nan 86.         34.
 51.         68.         64.         48.         89.         83.
 47.         65.27272727]

 Calculated Ensemble Mean:
65.27272727272727
Data